In [ ]:
import pandas as pd
import numpy as np
import wfdb
import ast
import os

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt

# Data Loading and Preprocessing Functions
def load_raw_data(df, sampling_rate, path):
    if sampling_rate == 100:
        data = [wfdb.rdsamp(path+f) for f in df.filename_lr]
    else:
        data = [wfdb.rdsamp(path+f) for f in df.filename_hr]
    data = np.array([signal for signal, meta in data])
    return data

def normalize_signals(X):
    mean = np.mean(X, axis=1, keepdims=True)
    std = np.std(X, axis=1, keepdims=True)
    X_norm = (X - mean) / (std + 1e-8)
    return X_norm

def aggregate_diagnostic(y_dic):
    tmp = []
    for key in y_dic.keys():
        if key in agg_df.index:
            tmp.append(agg_df.loc[key].diagnostic_class)
    return list(set(tmp))

def split_data(X, Y):
    test_fold = 10
    X_train = X[Y.strat_fold != test_fold]
    y_train = Y[Y.strat_fold != test_fold]['label'].values
    X_test = X[Y.strat_fold == test_fold]
    y_test = Y[Y.strat_fold == test_fold]['label'].values
    return X_train, y_train, X_test, y_test

def create_dataloaders(X_train, y_train, X_test, y_test, batch_size=32):
    X_train_tensor = torch.tensor(X_train, dtype=torch.float32)  # Shape: (batch_size, seq_len, channels)
    y_train_tensor = torch.tensor(y_train, dtype=torch.long)
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
    y_test_tensor = torch.tensor(y_test, dtype=torch.long)
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    return train_loader, test_loader

# Load Data
path = '../dataset/'
sampling_rate = 100

Y = pd.read_csv(path + 'ptbxl_database.csv', index_col='ecg_id')
Y.scp_codes = Y.scp_codes.apply(lambda x: ast.literal_eval(x))
X = load_raw_data(Y, sampling_rate, path)

agg_df = pd.read_csv(path + 'scp_statements.csv', index_col=0)
agg_df = agg_df[agg_df.diagnostic == 1]
Y['diagnostic_superclass'] = Y.scp_codes.apply(aggregate_diagnostic)

# Filter and Encode Labels
has_superclass = Y['diagnostic_superclass'].apply(lambda x: len(x) > 0)
X = X[has_superclass.values]
Y = Y[has_superclass]

# Assign a single superclass label (choose the first one)
Y['superclass_label'] = Y['diagnostic_superclass'].apply(lambda x: x[0])

# Map superclasses to integer labels
superclasses = sorted(Y['superclass_label'].unique())
superclass_to_int = {k: v for v, k in enumerate(superclasses)}
int_to_superclass = {v: k for k, v in superclass_to_int.items()}
Y['label'] = Y['superclass_label'].map(superclass_to_int)
num_classes = len(superclasses)

print("Diagnostic superclasses and their corresponding labels:")
for k, v in superclass_to_int.items():
    print(f"{k}: {v}")

# Split Data
X_train, y_train, X_test, y_test = split_data(X, Y)

# Normalize Data
X_train = normalize_signals(X_train)
X_test = normalize_signals(X_test)

# Create Data Loaders
train_loader, test_loader = create_dataloaders(X_train, y_train, X_test, y_test)

# Model Definition
import torch
import torch.nn as nn

class LSTMClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes, bidirectional=False):
        super(LSTMClassifier, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.bidirectional = bidirectional

        # Define the LSTM layer
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            bidirectional=bidirectional
        )

        # Define the output layer
        direction = 2 if bidirectional else 1
        self.fc = nn.Linear(hidden_size * direction, num_classes)

    def forward(self, x):
        # x shape: (batch_size, seq_len, input_size)
        # Initialize hidden and cell states
        h0 = torch.zeros(self.num_layers * (2 if self.bidirectional else 1), x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers * (2 if self.bidirectional else 1), x.size(0), self.hidden_size).to(x.device)

        # Forward propagate LSTM
        out, _ = self.lstm(x, (h0, c0))  # out: (batch_size, seq_len, hidden_size * num_directions)

        # Take the output from the last time step
        out = out[:, -1, :]  # (batch_size, hidden_size * num_directions)

        # Pass through the fully connected layer
        out = self.fc(out)  # (batch_size, num_classes)
        return out

# Training Parameters
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print("Using device:", device)

# Model parameters
input_size = X_train.shape[2]  # Number of features (channels)
hidden_size = 128
num_layers = 2
num_classes = len(superclasses)
bidirectional = True  # Set to True for a bidirectional LSTM

# Initialize the model, loss function, and optimizer
model = LSTMClassifier(input_size, hidden_size, num_layers, num_classes, bidirectional).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training Function
from sklearn.metrics import accuracy_score, f1_score

def train_model(model, train_loader, test_loader, criterion, optimizer, epochs=20):
    for epoch in range(epochs):
        model.train()
        train_loss = 0.0
        for X_batch, y_batch in train_loader:
            # Move data to the appropriate device
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            
            # Forward pass
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            
            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
        avg_train_loss = train_loss / len(train_loader)
        
        # Evaluation
        model.eval()
        y_pred = []
        y_true = []
        with torch.no_grad():
            for X_batch, y_batch in test_loader:
                X_batch = X_batch.to(device)
                outputs = model(X_batch)
                _, predicted = torch.max(outputs, 1)
                y_pred.extend(predicted.cpu().numpy())
                y_true.extend(y_batch.numpy())
        acc = accuracy_score(y_true, y_pred)
        f1 = f1_score(y_true, y_pred, average='macro')
        
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {avg_train_loss:.4f}, Accuracy: {acc*100:.2f}%, F1 Score: {f1:.4f}")

# Train the Model
train_model(model, train_loader, test_loader, criterion, optimizer, epochs=20)